# Identify co-occurring CAZy families

Author: Emma EM Hobbs

This notebook identifies CAZy families that appear in the same species together, the frequency the CAZy families appear together. Presuming that there is an evolutionary advantage to both families being presing the the CAZome (hence generting a high frequency of co-occurring), therefore, this approach could be used to identify CAZy families that may be frequently present in the same CAZyme together.

## Imports

In [41]:
import pandas as pd

from collections import Counter
from copy import copy

from tqdm.notebook import tqdm

## Load data

Load in the data contained the CSV file `genbank_acc_multi_fams.csv`. The CSV was generated by querying the local CAZyme database to retrieve the NCBI (GenBank) protein version accessions from the local CAZyme database, and their associated CAZy family annotations, for proteins associated with more than one family in the local CAZyme database (the CSV file was generated using the bash script `get_gbk_multi_fams.csv`.

Owing to the size of the file (which contains 398,182 lines), `get_gbk_multi_fams.csv` is read in chunks and parsed into a dictionary. Each row in the CSV file contains a unique family-protein accession pair, therefore, a protein accession will be present in multiple lines. Each key in the dictionary was a NCBI protein version accession, which was paired with the set of CAZy family annotations for the protein and that were retrieved from local CAZyme database.

In [4]:
# load in df with each pair of genbank accessions and cazy family annotation on a unique row
gbk_fam_dict = {}  # {gbk_protein_acc: {families}}

all_families = set()

print('Parsing df')
for chunk in tqdm(
    pd.read_csv("../Data/cooccuring_families/genbank_acc_multi_fams.csv", chunksize=1),
    desc="Parsing acc and fams",
    total=398183,
):
    # each chunk is one row

    acc = list(chunk['genbank_accession'])[0]  # protein version accession
    fam = list(chunk['family'])[0]
    
    # check if protein is already listed in the dictionary
    try:
        gbk_fam_dict[acc].add(fam)  # if in dict, add the CAZy family to the protein
        
    except KeyError:  # protein not listed in dict, so add protein
        gbk_fam_dict[acc] = {fam}
        
    all_families.add(fam)
        
print(
    f'Parsed df, which included {len(list(gbk_fam_dict.keys()))} '
    f'protein version accessions and {len(all_families)} CAZy families')

Parsing df


Parsing acc and fams:   0%|          | 0/398183 [00:00<?, ?it/s]

Parsed df, which included 191838 protein version accessions and 360 CAZy families


Formating dictionary:   0%|          | 0/191838 [00:00<?, ?it/s]

In [14]:
# convert sets to lists and order to faciliate comparison across the proteins
for acc in tqdm(gbk_fam_dict, desc='Formating dictionary'):
    families = list(gbk_fam_dict[acc])
    families.sort()
    
    gbk_fam_dict[acc] = str(families)

Formating dictionary:   0%|          | 0/191838 [00:00<?, ?it/s]

## Calculate incidence

Use the [`Counter` object](https://docs.python.org/3/library/collections.html?highlight=counter#collections.Counter) from the Python package `collections` to count the frequency that each group of CAZy families is listed in `gbk_fam_dict`. This will calculate how many proteins contain each group of CAZy families, and thus can be used to identify groups of CAZy families that frequently appear in the same CAZyme.

In [16]:
counter = Counter(gbk_fam_dict.values())
counter

Counter({"['CBM48', 'GH13']": 45737,
         "['CBM1', 'GH7']": 252,
         "['CBM20', 'GH13']": 1129,
         "['PL5', 'PL7']": 3,
         "['CBM59', 'GH26']": 21,
         "['CBM3', 'GH10', 'GH5']": 6,
         "['AA5', 'CBM32']": 146,
         "['GH13', 'GH133']": 409,
         "['CBM18', 'GH19']": 1170,
         "['CBM13', 'GH10']": 298,
         "['CBM34', 'GH13']": 8244,
         "['CBM50', 'GH25']": 1307,
         "['CBM3', 'GH9']": 422,
         "['CBM2', 'GH11']": 224,
         "['CBM22', 'CBM9', 'GH10']": 183,
         "['CBM5', 'GH5']": 113,
         "['CBM3', 'GH44']": 5,
         "['CBM17', 'CBM28', 'GH5']": 54,
         "['CBM3', 'GH5']": 855,
         "['CBM46', 'GH5']": 210,
         "['GH26', 'GH5']": 15,
         "['CBM26', 'GH13']": 630,
         "['CBM2', 'GH6']": 702,
         "['CBM2', 'CBM3', 'GH9']": 125,
         "['CBM4', 'GH9']": 354,
         "['CBM2', 'GH5']": 1472,
         "['CBM20', 'CBM34', 'GH13']": 72,
         "['CBM20', 'GH14']": 153,
         

The `Counter` object is not easy to read. Therefore, parse the `Counter` object into a dataframe, listing the group of CAZy families and the incidence (specifically, the number of protein accessions associated with this group in the local CAZyme database).

In [30]:
# sort the fam pairs into descending order
ordered_counter = {key: val for key, val in sorted(counter.items(), key = lambda ele: ele[1], reverse = True)}

# create a df of fam1, fam2, freq
cooccurring_fam_data = []

for families in tqdm(ordered_counter, desc='Building coccurring df'):
    freq = ordered_counter[families]
    fams = families.replace('[','').replace(']','').replace("'", "").replace(',','').replace(' ', '+').strip()
    
    cooccurring_fam_data.append([fams, freq])

# write to csv
cooccur_fams_df = pd.DataFrame(cooccurring_fam_data, columns=['Families', 'Incidence'])
cooccur_fams_df.to_csv('../Data/cooccuring_families/cooccurring_fams.csv')
cooccur_fams_df

Building coccurring df:   0%|          | 0/2234 [00:00<?, ?it/s]

,Families,Incidence
0,CBM48+GH13,45737
1,CBM50+GH23,14178
2,CBM91+GH43,9730
3,CBM34+GH13,8244
4,CBM5+GH18,5750
...,...,...
2229,CBM0+CBM35+GH39,1
2230,CE3+GH43,1
2231,CBM5+GH59,1
2232,CBM35+CBM47+GH107,1


In the above dataframe, `Families` lists the CAZy families that appear together in the same CAZyme (each CAZyme was identified by its unique NCBI protein version accession). The `Incidence` is the number of CAZymes (specifically, the number of unique NCBI protein version accessions) that contained all CAZy families in the group of families.

## Identify CAZymes with multiple catalytic domains

Most of the groups of CAZy families contained a Carbohydrate Binding Module (CBM) and a catalytic domain. CBMs are non-catalytic domains that facilitate the enzyme targeting and/or binding its substrate.

The interest was in in CAZymes containing more than one catalytic domain. Therefore, iterate through `coocur_fams_df` and identify rows containing CAZymes with multiple catalytic domains.

In [38]:
multi_cat_domains_data = []

for ri in tqdm(range(len(cooccur_fams_df)), desc="Identify groups with multiple catalytic domains"):
    families = cooccur_fams_df.iloc[ri]['Families'].split('+')
    
    num_of_fams_in_group = len(families)
    
    num_of_catalytic_domains = 0
    for fam in families:
        if fam.startswith('CBM') is False:  # Does not start with CBM, therefore Not a CBM domain
            # CBM domains are non-catalytic
            # all other domains are catalytic domains
            num_of_catalytic_domains += 1
            
    multi_cat_domains_data.append( [
        cooccur_fams_df.iloc[ri]['Families'],
        cooccur_fams_df.iloc[ri]['Incidence'],
        num_of_fams_in_group,
        num_of_catalytic_domains,
    ] )

cat_domains_df = pd.DataFrame(multi_cat_domains_data, 
                                   columns = [
                                       'Families',
                                       'Incidence',
                                       'Num_of_fams',
                                       'Num_of_catalytic_domains'
                                   ])
cat_domains_df

Identify groups with multiple catalytic domains:   0%|          | 0/2234 [00:00<?, ?it/s]

,Families,Incidence,Num_of_fams,Num_of_catalytic_domains
0,CBM48+GH13,45737,2,1
1,CBM50+GH23,14178,2,1
2,CBM91+GH43,9730,2,1
3,CBM34+GH13,8244,2,1
4,CBM5+GH18,5750,2,1
...,...,...,...,...
2229,CBM0+CBM35+GH39,1,3,1
2230,CE3+GH43,1,2,2
2231,CBM5+GH59,1,2,1
2232,CBM35+CBM47+GH107,1,3,1


Write out the groups of CAZy families that contain more that one catalytic family, as well the number of CAZymes (identified as the number of unqiue NCBI protein accessions) annotated with the corresponding group of families, the total number of families in the group and the number of catalytic families in the group.

In [40]:
multi_cat_domains_df = cat_domains_df[cat_domains_df['Num_of_catalytic_domains'] > 1]
multi_cat_domains_df.to_csv("../Data/cooccuring_families/multi_cat_domains_groups.csv")
multi_cat_domains_df

,Families,Incidence,Num_of_fams,Num_of_catalytic_domains
5,CE4+GH153,4586,2,2
6,GT2+GT4,4244,2,2
10,GH94+GT84,2266,2,2
16,GH17+GT2,1412,2,2
18,GT0+GT2,1305,2,2
...,...,...,...,...
2225,CBM3+GH5+PL9,1,3,2
2226,CBM3+GH12+GH48+GH5,1,4,3
2227,CBM3+PL11+PL9,1,3,2
2230,CE3+GH43,1,2,2


## Identify families that are subdivided

The analysis above looks at all CAZy families listed in CAZy.

To identify pairs of CAZy families whose co-occurrence could be represented by a CAZy subfamily, we could filter out CAZy families that are already subdivided into CAZy subfamilies.

The local CAZyme database `cazycsj` was queried using the bash script `get_fam_subfams.sh`, which generated the CSV file `cazy_fam_subfams.csv`, which lists every pair of CAZy family and associated CAZy subfamily.

In [42]:
cazy_fam_subfams_df = pd.read_csv('../Data/cooccuring_families/cazy_fam_subfams.csv')
cazy_fam_subfams_df.head(3)

,family,subfamily
0,AA10,NaN
1,CBM9,NaN
2,GH16,GH16_24


Itereate through `cazy_fam_subfams_df` and aggregate together all subfamilies for each CAZy family, then identify which families are not associated with any subfamilies. Write the names of these families to a new list.

In [43]:
cazy_fam_dict = {}  # {fam: {subfams}}

for ri in tqdm(range(len(cazy_fam_subfams_df)), desc='Gathering subfamilies'):
    row = cazy_fam_subfams_df.iloc[ri]  # select the row of the dataframe with the rowindex (ri)
    
    fam = row['family']
    subfam = row['subfamily']
    
    # check if the family is already in the dict
    try:
        cazy_fam_dict[fam].add(subfam)  # if present add subfam to set of subfams
    except KeyError:
        # add fam to the cazy_fam_dict
        cazy_fam_dict[fam] = {subfam}

# print example output
print('Subfamilies for AA10:', cazy_fam_dict['AA10'])
print('Subfamilies for GH16:', cazy_fam_dict['GH16'])

Gathering subfamilies:   0%|          | 0/745 [00:00<?, ?it/s]

Subfamilies for AA10: {nan}
Subfamilies for GH16: {nan, 'GH16_27', 'GH16_13', 'GH16_12', 'GH16_1', 'GH16_4', 'GH16_15', 'GH16_7', 'GH16_6', 'GH16_18', 'GH16_20', 'GH16_19', 'GH16_11', 'GH16_23', 'GH16_16', 'GH16_14', 'GH16_24', 'GH16_3', 'GH16_8', 'GH16_21', 'GH16_26', 'GH16_10', 'GH16_9', 'GH16_2', 'GH16_25', 'GH16_22', 'GH16_5', 'GH16_17'}


Identify families that have and have not been divided into subfamilies.

In [44]:
fams_wo_subfams = []  # families without subfams
fams_with_subfams = []  # families with subfams

for fam in cazy_fam_dict:
    if len(cazy_fam_dict[fam]) == 1:
        if (str(cazy_fam_dict[fam])) == '{nan}':
            fams_wo_subfams.append(fam)
    else:
        fams_with_subfams.append(fam)
        
        
fams_with_subfams = set(fams_with_subfams)  # remove duplicates from list if present
with open('fams_with_subfams', 'w') as fh:
    for fam in fams_with_subfams:
        fh.write(f"{fam}\n")
        
fams_wo_subfams = set(fams_wo_subfams)  # remove duplicates from list if present
with open('fams_withOUT_subfams', 'w') as fh:
    for fam in fams_wo_subfams:
        fh.write(f"{fam}\n")
        
print(
    f"{len(fams_with_subfams)} families are subdivided into subfamilies and "
    f"{len(fams_wo_subfams)} are NOT divided into subfamilies"
)

33 families are subdivided into subfamilies and 420 are NOT divided into subfamilies
